##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom training: walkthrough
# カスタム訓練：ウォークスルー

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/eager/custom_training_walkthrough"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/beta/tutorials/eager/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/beta/tutorials/eager/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ja/beta/tutorials/eager/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This guide uses machine learning to *categorize* Iris flowers by species. It uses TensorFlow to:

このガイドでは機械学習を使ってアヤメの花を種によって*分類*します。TensorFlowを使って下記のことを行います。

1. Build a model,
2. Train this model on example data, and
3. Use the model to make predictions about unknown data.

1. モデルを構築し、
2. このモデルをサンプルデータを使って訓練し、
3. このモデルを使って未知のデータに対する予測を行う

## TensorFlow programming
## TensorFlow プログラミング

This guide uses these high-level TensorFlow concepts:

このガイドでは、次のような TensorFlow の高レベルの概念を用います。

* Use TensorFlow's default [eager execution](https://www.tensorflow.org/guide/eager) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets),
* Build models and layers with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/).

* TensorFlow の既定の [Eager Execution](https://www.tensorflow.org/guide/eager) 開発環境を用いて、
* データを [Datasets API](https://www.tensorflow.org/guide/datasets) を使ってインポートし、
* TensorFlow の [Keras API](https://keras.io/getting-started/sequential-model-guide/) を使ってモデルと層を構築する

This tutorial is structured like many TensorFlow programs:

このチュートリアルは、多くの TensorFlow のプログラムと同様に下記のように構成されています。

1. Import and parse the dataset.
2. Select the type of model.
3. Train the model.
4. Evaluate the model's effectiveness.
5. Use the trained model to make predictions.

1. データセットのインポートとパース
2. モデルのタイプの選択
3. モデルの訓練
4. モデルの有効性の評価
5. 訓練済みモデルを使った予測

## Setup program
## プログラムの設定

### Configure imports
### インポートの構成

Import TensorFlow and the other required Python modules. By default, TensorFlow uses [Eager Execution](https://www.tensorflow.org/guide/eager) to evaluate operations immediately, returning concrete values instead of creating a [computational graph](https://www.tensorflow.org/guide/graphs) that is executed later. If you are used to a REPL or the `python` interactive console, this feels familiar.

TensorFlow とその他必要な Python のモジュールをインポートします。TensorFlow は既定で [Eager Execution](https://www.tensorflow.org/guide/eager) を使って演算結果を即時に評価し、後で実行される [computational graph（計算グラフ）](https://www.tensorflow.org/guide/graphs) を構築する代わりに具体的な値を返します。REPL や `python` の対話的コンソールを使っている方にとっては、馴染みのあるものです。

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import matplotlib.pyplot as plt

In [ ]:
!pip install tensorflow==2.0.0-beta1
import tensorflow as tf

In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

## The Iris classification problem
## アヤメ分類問題

Imagine you are a botanist seeking an automated way to categorize each Iris flower you find. Machine learning provides many algorithms to classify flowers statistically. For instance, a sophisticated machine learning program could classify flowers based on photographs. Our ambitions are more modest—we're going to classify Iris flowers based on the length and width measurements of their [sepals](https://en.wikipedia.org/wiki/Sepal) and [petals](https://en.wikipedia.org/wiki/Petal).

あなたが植物学者で、発見したアヤメの花を分類する自動的な方法を探しているとします。機械学習には花を統計的に分類するための方法がたくさんあります。たとえば、高度な機械学習プログラムを使うと、写真を元に花を分類することができます。私達の願望はもう少し控えめです。アヤメの花を [sepals（萼片）](https://en.wikipedia.org/wiki/Sepal) と [petals（花弁）](https://en.wikipedia.org/wiki/Petal) の長さと幅を使って分類することにしましょう。

The Iris genus entails about 300 species, but our program will only classify the following three:

アヤメ属にはおよそ300の種がありますが、ここで扱うプログラムは次の3つの種類のみを分類します。

* Iris setosa
* Iris virginica
* Iris versicolor

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/iris_three_species.jpg"
         alt="Petal geometry compared for three iris species: Iris setosa, Iris virginica, and Iris versicolor">
  </td></tr>
  <tr><td align="center">
    <b>図1.</b> <a href="https://commons.wikimedia.org/w/index.php?curid=170298">Iris setosa</a> (by <a href="https://commons.wikimedia.org/wiki/User:Radomil">Radomil</a>, CC BY-SA 3.0), <a href="https://commons.wikimedia.org/w/index.php?curid=248095">Iris versicolor</a>, (by <a href="https://commons.wikimedia.org/wiki/User:Dlanglois">Dlanglois</a>, CC BY-SA 3.0), and <a href="https://www.flickr.com/photos/33397993@N05/3352169862">Iris virginica</a> (by <a href="https://www.flickr.com/photos/33397993@N05">Frank Mayfield</a>, CC BY-SA 2.0).<br/>&nbsp;
  </td></tr>
</table>

Fortunately, someone has already created a [dataset of 120 Iris flowers](https://en.wikipedia.org/wiki/Iris_flower_data_set) with the sepal and petal measurements. This is a classic dataset that is popular for beginner machine learning classification problems.

幸いにして、萼片と花弁の計測値を使った [120のアヤメのデータセット](https://en.wikipedia.org/wiki/Iris_flower_data_set) を作ってくれた方がいます。これは初歩の機械学習による分類問題でよく使われる古典的なデータセットです。

## Import and parse the training dataset
## 訓練用データセットのインポートとパース

Download the dataset file and convert it into a structure that can be used by this Python program.

データセットファイルをダウンロードし、この Python プログラムで使えるような構造に変換します。

### Download the dataset
### データセットのダウンロード

Download the training dataset file using the [tf.keras.utils.get_file](https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file) function. This returns the file path of the downloaded file:

[tf.keras.utils.get_file](https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file) 関数を使って訓練用データセットをダウンロードします。この関数は、ダウンロードしたファイルのパスを返します。

In [ ]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

### Inspect the data
### データの観察

This dataset, `iris_training.csv`, is a plain text file that stores tabular data formatted as comma-separated values (CSV). Use the `head -n5` command to take a peek at the first five entries:

このデータセット `iris_training.csv` は、表形式のデータをコンマ区切り値（CSV）形式で格納したプレインテキストファイルです。`head -n5` コマンドを使って最初の5つのエントリを見てみましょう。

In [ ]:
!head -n5 {train_dataset_fp}

From this view of the dataset, notice the following:

上記のようにデータセットを見ることで次のことがわかります。

1. The first line is a header containing information about the dataset:
  * There are 120 total examples. Each example has four features and one of three possible label names.
2. Subsequent rows are data records, one *[example](https://developers.google.com/machine-learning/glossary/#example)* per line, where:
  * The first four fields are *[features](https://developers.google.com/machine-learning/glossary/#feature)*: these are the characteristics of an example. Here, the fields hold float numbers representing flower measurements.
  * The last column is the *[label](https://developers.google.com/machine-learning/glossary/#label)*: this is the value we want to predict. For this dataset, it's an integer value of 0, 1, or 2 that corresponds to a flower name.

1. 最初の行はデータセットに関する情報を含むヘッダである
  * 全体で120のサンプルがある。各サンプルには4つの特徴量があり、3つのラベルの1つが当てはまる。
2. 続く行はデータレコードで、1行が1つの *[example（サンプル）](https://developers.google.com/machine-learning/glossary/#example)* である。ここでは、
  * 最初の4つのフィールドはサンプルの特徴を示す *[features（特徴量）](https://developers.google.com/machine-learning/glossary/#feature)* である。ここで各フィールドは花の計測値を表す浮動小数点値をもつ。
  * 最後の列は予想の対象となる *[label（ラベル）](https://developers.google.com/machine-learning/glossary/#label)* である。このデータセットでは、花の名前に対応する整数値、0、1、2 のいずれかである。


Let's write that out in code:

コードを書いてみましょう。

In [ ]:
# column order in CSV file
# CSV ファイルの列の順序
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Each label is associated with string name (for example, "setosa"), but machine learning typically relies on numeric values. The label numbers are mapped to a named representation, such as:

ラベルはそれぞれ文字列の名前（たとえば、"setosa"）に関連付けられていますが、機械学習は数値に依存している場合がほとんどです。ラベルの数値は名前付きの表現にマップされます。たとえば次のようになります。

* `0`: Iris setosa
* `1`: Iris versicolor
* `2`: Iris virginica

For more information about features and labels, see the [ML Terminology section of the Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/framing/ml-terminology).

特徴量とラベルについてもっと知りたい場合には、 [ML Terminology section of the Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/framing/ml-terminology) を参照してください。

In [ ]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

### Create a `tf.data.Dataset`
### `tf.data.Dataset` の作成

TensorFlow's [Dataset API](https://www.tensorflow.org/guide/datasets) handles many common cases for loading data into a model. This is a high-level API for reading data and transforming it into a form used for training. See the [Datasets Quick Start guide](https://www.tensorflow.org/get_started/datasets_quickstart) for more information.

TensorFlow の [Dataset API](https://www.tensorflow.org/guide/datasets) はデータのモデルへのロードについて、多くの一般的なケースを扱います。この API は、データを読み込んで訓練に使われる形態へ変換するための、高レベル API です。詳しくは、 [Datasets Quick Start guide](https://www.tensorflow.org/get_started/datasets_quickstart) を参照してください。


Since the dataset is a CSV-formatted text file, use the [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset) function to parse the data into a suitable format. Since this function generates data for training models, the default behavior is to shuffle the data (`shuffle=True, shuffle_buffer_size=10000`), and repeat the dataset forever (`num_epochs=None`). We also set the [batch_size](https://developers.google.com/machine-learning/glossary/#batch_size) parameter:

今回のデータセットは CSV 形式のテキストファイルなので、データをパースして適切なフォーマットに変換するため、 [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset) 関数を使います。この関数はモデルの訓練のためのデータを生成するので、既定の動作はデータをシャッフルし (`shuffle=True, shuffle_buffer_size=10000`) 、データセットを永遠に繰り返すこと (`num_epochs=None`) です。また、 [batch_size](https://developers.google.com/machine-learning/glossary/#batch_size) パラメータも設定します。

In [ ]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

The `make_csv_dataset` function returns a `tf.data.Dataset` of `(features, label)` pairs, where `features` is a dictionary: `{'feature_name': value}`

`make_csv_dataset` 関数は、 `(features, label)` というペアの `tf.data.Dataset` を返します。ここで、 `features` は、 `{'feature_name': value}` というディクショナリです。

These `Dataset` objects are iterable. Let's look at a batch of features:

この `Dataset` オブジェクトはイテラブルです。特徴量のバッチを1つ見てみましょう。

In [ ]:
features, labels = next(iter(train_dataset))

print(features)

Notice that like-features are grouped together, or *batched*. Each example row's fields are appended to the corresponding feature array. Change the `batch_size` to set the number of examples stored in these feature arrays.

「特徴量のようなもの」はグループ化、言い換えると *バッチ化* されることに注意してください。それぞれのサンプル行のフィールドは、対応する特徴量の配列に追加されます。これらの特徴量配列に保存されるサンプルの数を設定するには、 `batch_size` を変更します。

You can start to see some clusters by plotting a few features from the batch:

上記のバッチから特徴量のいくつかをプロットしてみると、いくつかのクラスタが見られます。

In [ ]:
plt.scatter(features['petal_length'],
            features['sepal_length'],
            c=labels,
            cmap='viridis')

plt.xlabel("Petal length")
plt.ylabel("Sepal length")
plt.show()

To simplify the model building step, create a function to repackage the features dictionary into a single array with shape: `(batch_size, num_features)`.

モデル構築のステップを単純化するため、特徴量のディクショナリを `(batch_size, num_features)` という形状の単一の配列にパッケージし直す関数を作成します。

This function uses the [tf.stack](https://www.tensorflow.org/api_docs/python/tf/stack) method which takes values from a list of tensors and creates a combined tensor at the specified dimension:

この関数は、テンソルのリストから値を受け取り、指定された次元で結合されたテンソルを作成します。

In [ ]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  """特徴量を1つの配列にパックする"""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

Then use the [tf.data.Dataset.map](https://www.tensorflow.org/api_docs/python/tf/data/dataset/map) method to pack the `features` of each `(features,label)` pair into the training dataset:

次に、 [tf.data.Dataset.map](https://www.tensorflow.org/api_docs/python/tf/data/dataset/map) メソッドを使って `(features,label)` ペアそれぞれの `features` を訓練用データセットにパックします。

In [ ]:
train_dataset = train_dataset.map(pack_features_vector)

The features element of the `Dataset` are now arrays with shape `(batch_size, num_features)`. Let's look at the first few examples:

この `Dataset` の特徴量要素は、いまは形状 `(batch_size, num_features)` の配列になっています。 最初のサンプルをいくつか見てみましょう。

In [ ]:
features, labels = next(iter(train_dataset))

print(features[:5])

## Select the type of model
## モデルのタイプの選択

### Why model?
### なぜモデルか？

A *[model](https://developers.google.com/machine-learning/crash-course/glossary#model)* is a relationship between features and the label.  For the Iris classification problem, the model defines the relationship between the sepal and petal measurements and the predicted Iris species. Some simple models can be described with a few lines of algebra, but complex machine learning models have a large number of parameters that are difficult to summarize.

*[model（モデル）](https://developers.google.com/machine-learning/crash-course/glossary#model)* は特徴量とラベルの間の関係です。アヤメの分類問題の場合、モデルは萼片と花弁の計測値と予測されるアヤメの種類の関係を定義します。単純なモデルであれば、2、3行の数式で記述できますが、複雑な機械学習モデルには多数のパラメータがあり、かんたんに要約できるものではありません。

Could you determine the relationship between the four features and the Iris species *without* using machine learning?  That is, could you use traditional programming techniques (for example, a lot of conditional statements) to create a model?  Perhaps—if you analyzed the dataset long enough to determine the relationships between petal and sepal measurements to a particular species. And this becomes difficult—maybe impossible—on more complicated datasets. A good machine learning approach *determines the model for you*. If you feed enough representative examples into the right machine learning model type, the program will figure out the relationships for you.

機械学習を*使わずに*これらの4つの特徴量とアヤメの品種の関係を決定することは可能でしょうか？言い換えると、従来のプログラミング技術（たとえば、たくさんの条件文）を使ってモデルを作ることは可能でしょうか？おそらく、十分に時間をかけてデータセットを分析して、萼片と花弁の計測値と特定の品種との関係を決定すれば可能でしょう。もっと複雑なデータセットの場合には、これは困難であり、おそらく不可能です。機械学習を使ったよいアプローチでは、*あなたに代わってモデルを決定*してくれます。適切なタイプの機械学習モデルに、十分な量の典型的なサンプルを投入すれば、プログラムがあなたのためにこの関係性をみつけ出してくれるでしょう。

### Select the model
###  モデルの選択

We need to select the kind of model to train. There are many types of models and picking a good one takes experience. This tutorial uses a neural network to solve the Iris classification problem. *[Neural networks](https://developers.google.com/machine-learning/glossary/#neural_network)* can find complex relationships between features and the label. It is a highly-structured graph, organized into one or more *[hidden layers](https://developers.google.com/machine-learning/glossary/#hidden_layer)*. Each hidden layer consists of one or more *[neurons](https://developers.google.com/machine-learning/glossary/#neuron)*. There are several categories of neural networks and this program uses a dense, or *[fully-connected neural network](https://developers.google.com/machine-learning/glossary/#fully_connected_layer)*: the neurons in one layer receive input connections from *every* neuron in the previous layer. For example, Figure 2 illustrates a dense neural network consisting of an input layer, two hidden layers, and an output layer:

訓練すべきモデルの種類を決める必要があります。モデルにはたくさんの種類があり、よいものを選択するには経験が必要です。このチュートリアルでは、アヤメの分類問題を解くために、ニューラルネットワークを使用します。*[Neural networks（ニューラルネットワーク）](https://developers.google.com/machine-learning/glossary/#neural_network)* は、特徴量とラベルの間の複雑な関係をみつけることができます。ニューラルネットワークは、1つかそれ以上の *[hidden layers（隠れ層）](https://developers.google.com/machine-learning/glossary/#hidden_layer)* に構成された高度なグラフ構造です。それぞれの隠れ層には1つかそれ以上の *[neurons（ニューロン）](https://developers.google.com/machine-learning/glossary/#neuron)* があります。ニューラルネットワークにはいくつかのカテゴリーがありますが、このプログラムでは、密な、あるいは、 *[fully-connected neural network（全結合ニューラルネットワーク）](https://developers.google.com/machine-learning/glossary/#fully_connected_layer)* を使用します。全結合ニューラルネットワークでは、1つの層の中のニューロンすべてが、前の層の*すべての*ニューロンからの入力を受け取ります。例として、図2に入力層、2つの隠れ層、そして、出力層からなる密なニューラルネットワークを図示します。

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/custom_estimators/full_network.png"
         alt="A diagram of the network architecture: Inputs, 2 hidden layers, and outputs">
  </td></tr>
  <tr><td align="center">
    <b>図2.</b> 特徴量と隠れ層、予測をもつニューラルネットワーク<br/>&nbsp;
  </td></tr>
</table>

When the model from Figure 2 is trained and fed an unlabeled example, it yields three predictions: the likelihood that this flower is the given Iris species. This prediction is called *[inference](https://developers.google.com/machine-learning/crash-course/glossary#inference)*. For this example, the sum of the output predictions is 1.0. In Figure 2, this prediction breaks down as: `0.02` for *Iris setosa*, `0.95` for *Iris versicolor*, and `0.03` for *Iris virginica*. This means that the model predicts—with 95% probability—that an unlabeled example flower is an *Iris versicolor*.

図2のモデルが訓練されてラベルの付いていないサンプルを受け取ったとき、モデルは3つの予測値を返します。予測値はサンプルの花が与えられた3つの品種のそれぞれである可能性を示します。この予測は、 *[inference（推論）](https://developers.google.com/machine-learning/crash-course/glossary#inference)* とも呼ばれます。このサンプルでは、出力の予測値の合計は 1.0 になります。図2では、この予測値は *Iris setosa* が `0.02` 、 *Iris versicolor* が `0.95` 、 *Iris virginica* が `0.03` となっています。これは、モデルがこのラベルのない花を、95% の確率で *Iris versicolor* であると予測したことを意味します。

### Create a model using Keras
### Keras を使ったモデル構築

The TensorFlow [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras) API is the preferred way to create models and layers. This makes it easy to build models and experiment while Keras handles the complexity of connecting everything together.

TensorFlow の [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras) API は、モデルと層を作成するための好ましい方法です。Keras がすべてを結びつけるという複雑さを引き受けてくれるため、モデルや実験の構築がかんたんになります。

The [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model is a linear stack of layers. Its constructor takes a list of layer instances, in this case, two [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layers with 10 nodes each, and an output layer with 3 nodes representing our label predictions. The first layer's `input_shape` parameter corresponds to the number of features from the dataset, and is required:

[tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) モデルは層が一列に積み上げられたものです。このクラスのコンストラクタは、レイヤーインスタンスのリストを引数として受け取ります。今回の場合、それぞれ 10 のノードをもつ 2つの [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) レイヤーと、ラベルの予測値を表す3つのノードからなる出力レイヤーです。最初のレイヤーの `input_shape` パラメータがデータセットの特徴量の数に対応しており、これは必須パラメータです。

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

The *[activation function](https://developers.google.com/machine-learning/crash-course/glossary#activation_function)* determines the output shape of each node in the layer. These non-linearities are important—without them the model would be equivalent to a single layer. There are many [available activations](https://www.tensorflow.org/api_docs/python/tf/keras/activations), but [ReLU](https://developers.google.com/machine-learning/crash-course/glossary#ReLU) is common for hidden layers.

*[activation function（活性化関数）](https://developers.google.com/machine-learning/crash-course/glossary#activation_function)*  は、そのレイヤーの各ノードの出力の形を決定します。この関数の非線形性は重要であり、それがなければモデルは 1層しかないものと等価になってしまいます。[利用可能な活性化関数](https://www.tensorflow.org/api_docs/python/tf/keras/activations) はたくさんありますが、隠れ層では [ReLU](https://developers.google.com/machine-learning/crash-course/glossary#ReLU) が一般的です。

The ideal number of hidden layers and neurons depends on the problem and the dataset. Like many aspects of machine learning, picking the best shape of the neural network requires a mixture of knowledge and experimentation. As a rule of thumb, increasing the number of hidden layers and neurons typically creates a more powerful model, which requires more data to train effectively.

理想的な隠れ層の数やニューロンの数は問題やデータセットによって異なります。機械学習のさまざまな面と同様に、ニューラルネットワークの最良の形を選択するには、知識と経験の両方が必要です。経験則から、一般的には隠れ層やニューロンの数を増やすとより強力なモデルを作ることができますが、効果的に訓練を行うためにより多くのデータを必要とします。

### Using the model
### モデルの使用

Let's have a quick look at what this model does to a batch of features:

それでは、このモデルが特徴量のバッチに対して何を行うかを見てみましょう。

In [ ]:
predictions = model(features)
predictions[:5]

Here, each example returns a [logit](https://developers.google.com/machine-learning/crash-course/glossary#logits) for each class.

ご覧のように、サンプルのそれぞれは、各クラスの [logit（ロジット）](https://developers.google.com/machine-learning/crash-course/glossary#logits) 値を返します。


To convert these logits to a probability for each class, use the [softmax](https://developers.google.com/machine-learning/crash-course/glossary#softmax) function:

これらのロジット値を各クラスの確率に変換するためには、 [softmax](https://developers.google.com/machine-learning/crash-course/glossary#softmax) 関数を使用します。

In [ ]:
tf.nn.softmax(predictions[:5])

Taking the `tf.argmax` across classes gives us the predicted class index. But, the model hasn't been trained yet, so these aren't good predictions:

クラス間で `tf.argmax` を取ると、予測されたクラスのインデックスが得られます。しかし、このモデルはまだ訓練されていないので、予測はよいものではありません。

In [ ]:
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))

## Train the model
## モデルの訓練

*[Training](https://developers.google.com/machine-learning/crash-course/glossary#training)* is the stage of machine learning when the model is gradually optimized, or the model *learns* the dataset. The goal is to learn enough about the structure of the training dataset to make predictions about unseen data. If you learn *too much* about the training dataset, then the predictions only work for the data it has seen and will not be generalizable. This problem is called *[overfitting](https://developers.google.com/machine-learning/crash-course/glossary#overfitting)*—it's like memorizing the answers instead of understanding how to solve a problem.

*[Training（訓練）](https://developers.google.com/machine-learning/crash-course/glossary#training)* は、機械学習において、モデルが徐々に最適化されていく、あるいはモデルがデータセットを*学習する*段階です。目的は、見たことのないデータについて予測を行うため、訓練用データセットの構造を十分に学習することです。訓練用データセットを学習*しすぎる*と、予測は見たことのあるデータに対してしか有効ではなく、一般化できません。この問題は *[overfitting（過学習）](https://developers.google.com/machine-learning/crash-course/glossary#overfitting)* と呼ばれ、問題の解き方を理解するのではなく答えを丸暗記するようなものです。

The Iris classification problem is an example of *[supervised machine learning](https://developers.google.com/machine-learning/glossary/#supervised_machine_learning)*: the model is trained from examples that contain labels. In *[unsupervised machine learning](https://developers.google.com/machine-learning/glossary/#unsupervised_machine_learning)*, the examples don't contain labels. Instead, the model typically finds patterns among the features.

アヤメの分類問題は、 *[supervised machine learning（教師あり学習）](https://developers.google.com/machine-learning/glossary/#supervised_machine_learning)* の1種で、モデルはラベルの付いたサンプルを学習します。*[unsupervised machine learning（教師なし学習）](https://developers.google.com/machine-learning/glossary/#unsupervised_machine_learning)* では、サンプルにラベルはありません。その代わり、一般的にはモデルが特徴量からパターンを発見します。

### Define the loss and gradient function
### 損失関数と勾配関数の定義

Both training and evaluation stages need to calculate the model's *[loss](https://developers.google.com/machine-learning/crash-course/glossary#loss)*. This measures how off a model's predictions are from the desired label, in other words, how bad the model is performing. We want to minimize, or optimize, this value.

訓練段階と評価段階では、モデルの *[loss（損失）](https://developers.google.com/machine-learning/crash-course/glossary#loss)* を計算する必要があります。損失とは、モデルの予測が望ましいラベルからどれくらい離れているかを測定するものです。言い換えると、どれくらいモデルの性能が悪いかを示します。この値を最小化、または最適化したいのです。

Our model will calculate its loss using the `tf.keras.losses.SparseCategoricalCrossentropy` function which takes the model's class probability predictions and the desired label, and returns the average loss across the examples.

今回のモデルでは、 `tf.keras.losses.SparseCategoricalCrossentropy` 関数を使って損失を計算します。これは、モデルのクラスごとの予測確率と望まれるラベルを使って、サンプル間の平均損失を返します。

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y):
  y_ = model(x)

  return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels)
print("Loss test: {}".format(l))

Use the [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) context to calculate the *[gradients](https://developers.google.com/machine-learning/crash-course/glossary#gradient)* used to optimize your model:

[tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) コンテキストを使って、モデルを最適化する際に使われる *[gradients（勾配）](https://developers.google.com/machine-learning/crash-course/glossary#gradient)* を計算しましょう。

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

### Create an optimizer
### オプティマイザーの作成

An *[optimizer](https://developers.google.com/machine-learning/crash-course/glossary#optimizer)* applies the computed gradients to the model's variables to minimize the `loss` function. You can think of the loss function as a curved surface (see Figure 3) and we want to find its lowest point by walking around. The gradients point in the direction of steepest ascent—so we'll travel the opposite way and move down the hill. By iteratively calculating the loss and gradient for each batch, we'll adjust the model during training. Gradually, the model will find the best combination of weights and bias to minimize loss. And the lower the loss, the better the model's predictions.

*[optimizer](https://developers.google.com/machine-learning/crash-course/glossary#optimizer)* は、`loss` 関数を最小化するため、計算された勾配をモデルの変数に適用します。損失関数は、局面として考えることができ（図3 参照）、歩き回ることによって最小となる点をみつけたいのです。勾配は、一番急な上りの方向を示すため、逆方向に進んで丘を下ることになります。バッチごとに損失と勾配を計算することにより、訓練中のモデルを調節します。モデルは徐々に、損失を最小化する重みとバイアスの最適な組み合わせをみつけます。損失が小さいほど、モデルの予測がよくなります。

<table>
  <tr><td>
    <img src="https://cs231n.github.io/assets/nn3/opt1.gif" width="70%"
         alt="Optimization algorithms visualized over time in 3D space.">
  </td></tr>
  <tr><td align="center">
    <b>図3.</b> 3次元空間における最適化アルゴリズムの時系列可視化。<br/>(Source: <a href="http://cs231n.github.io/neural-networks-3/">Stanford class CS231n</a>, MIT License, Image credit: <a href="https://twitter.com/alecrad">Alec Radford</a>)
  </td></tr>
</table>

TensorFlow has many [optimization algorithms](https://www.tensorflow.org/api_guides/python/train) available for training. This model uses the [tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer) that implements the *[stochastic gradient descent](https://developers.google.com/machine-learning/crash-course/glossary#gradient_descent)* (SGD) algorithm. The `learning_rate` sets the step size to take for each iteration down the hill. This is a *hyperparameter* that you'll commonly adjust to achieve better results.

TensorFlow には、訓練に使える [optimization algorithms（最適化アルゴリズム）](https://www.tensorflow.org/api_guides/python/train) がたくさんあります。このモデルでは、*[stochastic gradient descent（確率的勾配降下法）](https://developers.google.com/machine-learning/crash-course/glossary#gradient_descent)* (SGD) を実装した [tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer) アルゴリズムを使用します。`learning_rate` （学習率）は、イテレーションごとに丘を下る際のステップのサイズを設定します。通常これはよりよい結果を得るために調整する *hyperparameter（ハイパーパラメータ）* です。 

Let's setup the optimizer:

オプティマイザーを設定しましょう。

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

We'll use this to calculate a single optimization step:

これを使って、最適化を１ステップ分計算してみます。

In [ ]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels).numpy()))

### Training loop
### 訓練ループ

With all the pieces in place, the model is ready for training! A training loop feeds the dataset examples into the model to help it make better predictions. The following code block sets up these training steps:

すべての部品が揃ったので、モデルの訓練ができるようになりました。訓練ループは、モデルにデータセットのサンプルを供給し、モデルがよりよい予測を行えるようにします。下記のコードブロックは、この訓練のステップを構成します。

1. Iterate each *epoch*. An epoch is one pass through the dataset.
2. Within an epoch, iterate over each example in the training `Dataset` grabbing its *features* (`x`) and *label* (`y`).
3. Using the example's features, make a prediction and compare it with the label. Measure the inaccuracy of the prediction and use that to calculate the model's loss and gradients.
4. Use an `optimizer` to update the model's variables.
5. Keep track of some stats for visualization.
6. Repeat for each epoch.

1. *epoch（エポック）* をひとつずつ繰り返します。エポックとは、データセットをひととおり処理するということです。
2. エポック内では、訓練用の `Dataset（データセット）` のサンプルひとつずつから、その *features（特徴量）* (`x`) と *label（ラベル）* (`y`) を取り出して処理します。
3. サンプルの特徴量を使って予測を行い、ラベルと比較します。予測の不正確度を測定し、それを使ってモデルの損失と勾配を計算します。
4. `optimizer` を使って、モデルの変数を更新します。
5. 可視化のためにいくつかの統計量を記録します。
6. これをエポックごとに繰り返します。

The `num_epochs` variable is the number of times to loop over the dataset collection. Counter-intuitively, training a model longer does not guarantee a better model. `num_epochs` is a *[hyperparameter](https://developers.google.com/machine-learning/glossary/#hyperparameter)* that you can tune. Choosing the right number usually requires both experience and experimentation:

`num_epochs` 変数は、データセットのコレクションを何回繰り返すかという数字です。直感には反しますが、モデルを長く訓練すれば必ずよいモデルが得られるというわけではありません。`num_epochs` は、*[hyperparameter（ハイパーパラメータ）](https://developers.google.com/machine-learning/glossary/#hyperparameter)* であり、チューニングできます。適切な数を選択するには、経験と実験の両方が必要です。


In [ ]:
## Note: Rerunning this cell uses the same model variables
## Note: このセルを再実行すると同じモデル変数が使われます

# Keep results for plotting
# 結果をグラフ化のために保存
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  # 訓練ループ - 32個ずつのバッチを使用
  for x, y in train_dataset:
    # Optimize the model
    # モデルの最適化
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    # 進捗の記録
    epoch_loss_avg(loss_value)  # Add current batch loss 現在のバッチの損失を加算
    # Compare predicted label to actual label
    # 予測ラベルと実際のラベルを比較
    epoch_accuracy(y, model(x))

  # End epoch
  # エポックの終わり
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

### Visualize the loss function over time
### 時間の経過に対する損失関数の可視化

While it's helpful to print out the model's training progress, it's often *more* helpful to see this progress. [TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is a nice visualization tool that is packaged with TensorFlow, but we can create basic charts using the `matplotlib` module.

モデルの訓練の進み方をプリントするのも役立ちますが、普通はこの進捗を見るほうが*もっと*役に立ちます。[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) は TensorFlow に付属する優れた可視化ツールですが、
 `matplotlib` モジュールを使って基本的なグラフを描画することもできます。
 
Interpreting these charts takes some experience, but you really want to see the *loss* go down and the *accuracy* go up:

これらのグラフを解釈するにはある程度の経験が必要ですが、*損失*が低下して、*正解率*が上昇するのをみたいですよね。 

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

## Evaluate the model's effectiveness
## モデルの有効性評価

Now that the model is trained, we can get some statistics on its performance.

モデルの訓練が終わったら、その性能の指標を得ることができます。

*Evaluating* means determining how effectively the model makes predictions. To determine the model's effectiveness at Iris classification, pass some sepal and petal measurements to the model and ask the model to predict what Iris species they represent. Then compare the model's predictions against the actual label.  For example, a model that picked the correct species on half the input examples has an *[accuracy](https://developers.google.com/machine-learning/glossary/#accuracy)* of `0.5`. Figure 4 shows a slightly more effective model, getting 4 out of 5 predictions correct at 80% accuracy:

*評価*とは、モデルの予測が効果的であるかどうかを決定することを言います。アヤメの分類でモデルの有効性を見定めるには、モデルに萼片と花弁の測定値をいくつか与え、どのアヤメの品種であるかを予測させます。そして、モデルの予測と実際のラベルを比較します。たとえば、あるモデルが入力サンプルの半分で正解の品種をえらんだとすると、*[accuracy（正解率）](https://developers.google.com/machine-learning/glossary/#accuracy)* は `0.5` ということになります。図４で示すモデルはもう少し効果的であり、5つの予測中4つで正解し、正解率は 80% です。

<table cellpadding="8" border="0">
  <colgroup>
    <col span="4" >
    <col span="1" bgcolor="lightblue">
    <col span="1" bgcolor="lightgreen">
  </colgroup>
  <tr bgcolor="lightgray">
    <th colspan="4">サンプルの特徴量</th>
    <th colspan="1">ラベル</th>
    <th colspan="1" >モデルの予測値</th>
  </tr>
  <tr>
    <td>5.9</td><td>3.0</td><td>4.3</td><td>1.5</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr>
    <td>6.9</td><td>3.1</td><td>5.4</td><td>2.1</td><td align="center">2</td><td align="center">2</td>
  </tr>
  <tr>
    <td>5.1</td><td>3.3</td><td>1.7</td><td>0.5</td><td align="center">0</td><td align="center">0</td>
  </tr>
  <tr>
    <td>6.0</td> <td>3.4</td> <td>4.5</td> <td>1.6</td> <td align="center">1</td><td align="center" bgcolor="red">2</td>
  </tr>
  <tr>
    <td>5.5</td><td>2.5</td><td>4.0</td><td>1.3</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr><td align="center" colspan="6">
    <b>図4.</b> 正解率 80％ のアヤメ分類器<br/>&nbsp;
  </td></tr>
</table>

### Setup the test dataset
### テスト用データセットの設定

Evaluating the model is similar to training the model. The biggest difference is the examples come from a separate *[test set](https://developers.google.com/machine-learning/crash-course/glossary#test_set)* rather than the training set. To fairly assess a model's effectiveness, the examples used to evaluate a model must be different from the examples used to train the model.

モデルの評価はモデルの訓練と同様です。もっとも大きな違いは、サンプルが訓練用データセットではなく*[test set（テスト用データセット）](https://developers.google.com/machine-learning/crash-course/glossary#test_set)* からのものであるという点です。モデルの有効性を正しく評価するには、モデルの評価に使うサンプルは訓練用データセットのものとは違うものでなければなりません。

The setup for the test `Dataset` is similar to the setup for training `Dataset`. Download the CSV text file and parse that values, then give it a little shuffle:

テスト用 `Dataset` の設定は、訓練用 `Dataset` の設定と同様です。CSV ファイルをダウンロードし、値をパースしてからすこしシャッフルを行います。

In [ ]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)

In [ ]:
test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

### Evaluate the model on the test dataset
### テスト用データセットでのモデルの評価

Unlike the training stage, the model only evaluates a single [epoch](https://developers.google.com/machine-learning/glossary/#epoch) of the test data. In the following code cell, we iterate over each example in the test set and compare the model's prediction against the actual label. This is used to measure the model's accuracy across the entire test set:

訓練段階とは異なり、モデルはテスト用データの 1[epoch](https://developers.google.com/machine-learning/glossary/#epoch) だけで行います。下記のコードセルでは、テスト用データセットのサンプルをひとつずつ処理し、モデルの予測と実際のラベルを比較します。この結果を使ってテスト用データセット全体でのモデルの正解率を計算します。

In [ ]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  logits = model(x)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

We can see on the last batch, for example, the model is usually correct:

たとえば最後のバッチを見ると、モデルはだいたい正確です。

In [ ]:
tf.stack([y,prediction],axis=1)

## Use the trained model to make predictions
## 訓練済みモデルを使った予測

We've trained a model and "proven" that it's good—but not perfect—at classifying Iris species. Now let's use the trained model to make some predictions on [unlabeled examples](https://developers.google.com/machine-learning/glossary/#unlabeled_example); that is, on examples that contain features but not a label.

モデルを訓練し、それが、完全ではないにせよアヤメの品種の分類でよい性能を示すことを「証明」しました。次は、この訓練済みモデルを使って [unlabeled examples（ラベルなしサンプル）](https://developers.google.com/machine-learning/glossary/#unlabeled_example) つまり、特徴量だけでラベルのないサンプルについて予測を行ってみましょう。

In real-life, the unlabeled examples could come from lots of different sources including apps, CSV files, and data feeds. For now, we're going to manually provide three unlabeled examples to predict their labels. Recall, the label numbers are mapped to a named representation as:

現実世界では、ラベルなしサンプルはアプリや CSV ファイル、データフィードなどさまざまな異なるソースからやってきます。ここでは、ラベルを予測するため、3つのラベルなしサンプルを手動で与えることにします。ラベルの番号は、下記のように名前を表していることを思い出してください。

* `0`: Iris setosa
* `1`: Iris versicolor
* `2`: Iris virginica

In [ ]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))